In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = str(1)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dataloader import *
from model import *
from utilities import *
from retrain_fun import *
import random
random.seed(0)
np.random.seed(0)
jax.random.PRNGKey(0)
os.environ["XLA_FLAGS"] = "--xla_gpu_deterministic_reductions --xla_gpu_autotune_level=2"
os.environ['TF_CUDNN_DETERMINISTIC']='1'


In [ ]:
data_aggregated, houses = data_preprocess(only_positive=True)

Train: [2361, 7719, 9019, 2335, 7951] 
Test:  [4373, 7901, 3456, 3538, 2818]

In [ ]:
train_list= [[2361, 7719, 9019, 2335, 7951]]
test_list= [[4373, 7901, 3456, 3538, 2818]]
pool_list=[]
for i in range(len(train_list)):
    pool = [j for j in houses if j not in train_list[i]+test_list[i]]
    pool_list.append(pool)

In [ ]:
appliance = 'refrigerator'

In [ ]:
pool_list

In [ ]:
total_rmse=[]
total_mae = []
houseids=[]
for i in range(len(train_list)):
    # print(len(train_list))
    print("Train Set No:", i+1)
    train_houses = train_list[i]
    test_houses = test_list[i]
    pool_houses = pool_list[i]
    train = data_aggregated[data_aggregated["dataid"].isin(train_houses)] 
    test = data_aggregated[data_aggregated["dataid"].isin(test_houses)]

    epistemic_list = []
    rmse_iter=[]
    mae_iter = []
    houses=[]
    start= ["2018-03-01 00:00:00-06","2018-03-11 00:00:00-06", "2018-03-16 00:00:00-06", "2018-03-21 00:00:00-06", "2018-03-26 00:00:00-06", "2018-04-01 00:00:00-06","2018-04-06 00:00:00-06","2018-04-11 00:00:00-06","2018-04-16 00:00:00-06","2018-04-21 00:00:00-06"]
    end= ["2018-03-10 23:59:00-06","2018-03-15 23:59:00-06","2018-03-20 23:59:00-06","2018-03-25 23:59:00-06","2018-03-31 23:59:00-06","2018-04-05 23:59:00-06", "2018-04-10 23:59:00-06", "2018-04-15 23:59:00-06", "2018-04-20 23:59:00-06", "2018-04-25 23:59:00-06"]
    dates = ["2018-03-01 00:00:00-06","2018-03-06 00:00:00-06", "2018-03-11 00:00:00-06", "2018-03-16 00:00:00-06", "2018-03-21 00:00:00-06", "2018-03-27 00:00:00-06","2018-04-01 00:00:00-06","2018-04-06 00:00:00-06","2018-04-11 00:00:00-06","2018-04-16 00:00:00-06"]
    for j in range(len(start)):
        # print(len(start))
        print("Iteration No: ",j)
        if(j==0):
            house, houseid,rmsee, train, test, maee, epistemic= retrain(0, start[j], end[j], pool_houses, train, test, dates[j], appliance, data_aggregated)
            rmse_iter.append(rmsee.item())
            mae_iter.append(maee.item())
            houses.append(house)
            epistemic_list.append(epistemic)
            pool_houses= np.delete(pool_houses, houseid)
        elif(j==10):
            house, houseid,rmsee, train, test, maee, epistemic= retrain(house, start[j], end[j], pool_houses, train, test, dates[j], appliance, data_aggregated)
            rmse_iter.append(rmsee.item())
            mae_iter.append(maee.item())
            epistemic_list.append(epistemic)
        else:
            house, houseid,rmsee, train, test, maee, epistemic= retrain(house, start[j], end[j], pool_houses, train, test, dates[j], appliance, data_aggregated)
            rmse_iter.append(rmsee.item())
            mae_iter.append(maee.item())
            houses.append(house)
            pool_houses= np.delete(pool_houses, houseid)
            epistemic_list.append(epistemic)
    total_rmse.append(rmse_iter)
    total_mae.append(mae_iter)
    houseids.append(houses)

In [ ]:
houseids

In [ ]:
total_rmse

In [ ]:
# d = pd.DataFrame(total_rmse)
# d2 = pd.DataFrame(total_mae)
# house_df= pd.DataFrame(houseids)
# if not os.path.exists(f'{appliance}'):
#     os.mkdir(f'{appliance}')

# # Save the DataFrame to a CSV file
# d.to_csv(os.path.join(f'{appliance}', "mi4_rmse.csv"), index=False)
# d2.to_csv(os.path.join(f'{appliance}', "mi4_mae.csv"), index=False)
# house_df.to_csv(os.path.join(f'{appliance}', "mi4houseids.csv"), index=False)

In [ ]:
plt.plot(total_rmse[0])

plt.xlabel("#Iterations")
plt.ylabel("RMSE")
plt.title("air")



In [ ]:
d = pd.DataFrame(epistemic_list)
d.to_csv('refrigerator_epistemic.csv')

In [ ]:
epistemic_list